In [ ]:
import pandas as pd 

: 

In [ ]:
file_path_stations = r"/Users/melvyn/Documents/maths pour l'info/projet metro/asset/stations.txt"
stations = pd.read_csv(file_path_stations, sep=";", header=0)
stations

In [ ]:
file_path_liaisons = r"/Users/melvyn/Documents/maths pour l'info/projet metro/asset/liaisons.txt"
liaisons = pd.read_csv(file_path_liaisons, sep=" ", header=0)
liaisons

pour chaque sommet : 
    pour chaque autre sommet : 
        verifier si un chemin existe 

parcours en largeur : 
    

In [ ]:
def generate_instructions(graph: dict, chemin: list, distances: dict) -> list:
    """
    Génère les instructions pour le chemin donné.
    
    Parameters:
    - graph (dict): Dictionnaire représentant le graphe.
    - chemin (list): Liste des sommets du chemin calculé.
    - distances (dict): Dictionnaire des distances depuis le sommet de départ.
    
    Returns:
    - list: Instructions textuelles du chemin.
    """
    instructions = []
    current_line = None
    station_count = 0
    segment_time = 0
    first = True 
    nom = [graph[chemin[i]]['nom_sommet'] for i in range(len(chemin))]
    print("nom : ", nom)
    for i in range(len(chemin) - 1):
        current = chemin[i]
        next_station = chemin[i + 1]
        
        current_station = graph[current]
        next_station_data = graph[next_station]
        
        # Première station
        if i == 0:
            instructions.append(f"Vous êtes à {current_station['nom_sommet']}.")
            ref_station = current
        if current_station["nom_sommet"] == "Champs Élysées, Clémenceau": 
            print(i)
        # Changement de ligne ou continuation sur la même ligne
        if current_station['numéro_ligne'] != next_station_data['numéro_ligne']:
            # Si on change de ligne, on ajoute les informations du segment précédent
            if current_line is not None and first:
                #terminus_direction = find_correct_direction(graph, current, next_station, current_line, chemin[-1])
                terminus_direction = find_correct_direction2(graph, current, ref_station, current_line)
                instructions.append(
                    f"Prenez la ligne {current_line} direction {terminus_direction} pour {station_count} stations, "
                    f"durée estimée {segment_time // 60} minutes."
                )
                first = False
            elif current_line is not None :  
                terminus_direction = find_correct_direction2(graph, current, ref_station, current_line)
                # Ajouter l'instruction de changement de ligne
                instructions.append(
                    f"A {graph[ref_station]['nom_sommet']}, changez et prenez la ligne "
                    f"{current_line} direction {terminus_direction} pour {station_count} stations, durée estimée {segment_time // 60} minutes."
                )
            # Trouver la direction de la nouvelle ligne
            #terminus_direction = find_correct_direction(graph, next_station, chemin[-1], next_station_data['numéro_ligne'], chemin[-1])
            #terminus_direction = find_correct_direction2(graph, next_station, chemin[-1], next_station_data['numéro_ligne'])
            
            # Ajouter l'instruction de changement de ligne
            #instructions.append(
            #    f"A {current_station['nom_sommet']}, changez et prenez la ligne "
            #    f"{next_station_data['numéro_ligne']} direction {terminus_direction} pour {station_count} stations, durée estimée {segment_time // 60} minutes."
            #)
            
            # Réinitialiser les compteurs pour le nouveau segment de ligne
            ref_station = current
            current_line = next_station_data['numéro_ligne']
            station_count = 1
            segment_time = next(voisin['temps_en_secondes'] for voisin in current_station['voisins'] if voisin['num_sommet'] == next_station)
            print("segment_time : ", segment_time)
        else:
            # Si on reste sur la même ligne, on incrémente les compteurs
            if current_line is None:
                current_line = current_station['numéro_ligne']
            station_count += 1
            segment_time += next(voisin['temps_en_secondes'] for voisin in current_station['voisins'] if voisin['num_sommet'] == next_station)
    
    # Ajouter les informations pour le dernier segment de ligne
    terminus_direction = find_correct_direction(graph, chemin[-2], chemin[-1], current_line, chemin[-1])
    instructions.append(
        f"Prenez la ligne {current_line} direction {terminus_direction} pour {station_count} stations, durée estimée {segment_time // 60} minutes."
    )
    
    # Temps total pour le trajet complet
    instructions.append(f"Vous devriez arriver à {graph[chemin[-1]]['nom_sommet']} en environ {distances[chemin[-1]] // 60} minutes.")
    
    return instructions


In [ ]:
def generate_instructions1(graph: dict, chemin: list, distances: dict) -> list:
     """
     Génère les instructions pour le chemin donné.
   
     Parameters:
     - graph (dict): Dictionnaire représentant le graphe.
     - chemin (list): Liste des sommets du chemin calculé.
     - distances (dict): Dictionnaire des distances depuis le sommet de départ.
   
     Returns:
     - list: Instructions textuelles du chemin.
     """
     instructions = []
     current_line = None
     station_count = 0
     segment_time = 0
   
     for i in range(len(chemin) - 1):
         current = chemin[i]
         next_station = chemin[i + 1]
       
         current_station = graph[current]
         next_station_data = graph[next_station]
       
         # Première station
         if i == 0:
             instructions.append(f"Vous êtes à {current_station['nom_sommet']}.")
         # Changement de ligne ou continuation sur la même ligne
         if current_station['numéro_ligne'] != next_station_data['numéro_ligne']:
             # Si on change de ligne, on ajoute les informations du segment précédent
             if current_line is not None:
                 instructions.append(
                     f"Prenez la ligne {current_line} pour {station_count} stations, "
                     f"durée estimée {segment_time // 60} minutes."
                 )
           
             # Trouver la direction de la nouvelle ligne
             direction_station = trouver_terminus_proche(graph, next_station, next_station_data['numéro_ligne'], distances)
           
             # Ajouter l'instruction de changement de ligne
             instructions.append(
                 f"A {current_station['nom_sommet']}, changez et prenez la ligne "
                 f"{next_station_data['numéro_ligne']} direction {direction_station}."
             )
           
             # Réinitialiser les compteurs pour le nouveau segment de ligne
             current_line = next_station_data['numéro_ligne']
             station_count = 1
             segment_time = next(voisin['temps_en_secondes'] for voisin in current_station['voisins'] if voisin['num_sommet'] == next_station)
         else:
             # Si on reste sur la même ligne, on incrémente les compteurs
             if current_line is None:
                 current_line = current_station['numéro_ligne']
             station_count += 1
             segment_time += next(voisin['temps_en_secondes'] for voisin in current_station['voisins'] if voisin['num_sommet'] == next_station)
   
     # Ajouter les informations pour le dernier segment de ligne
     instructions.append(
         f"Prenez la ligne {current_line} pour {station_count} stations, durée estimée {segment_time // 60} minutes."
     )
   
     # Temps total pour le trajet complet
     instructions.append(f"Vous devriez arriver à {graph[chemin[-1]]['nom_sommet']} dans environ {distances[chemin[-1]] // 60} minutes.")
   
     return instructions
def trouver_terminus_proche1(graph: dict, station_actuelle: int, ligne: int, distances: dict) -> str:
     """
     Trouve le terminus le plus proche sur une ligne donnée pour une station actuelle.
   
     Parameters:
     - graph (dict): Dictionnaire représentant le graphe.
     - station_actuelle (int): Le sommet actuel.
     - ligne (int): Le numéro de la ligne.
     - distances (dict): Dictionnaire des distances calculées pour chaque sommet.
   
     Returns:
     - str: Le nom du terminus le plus proche.
     """
     terminus_candidates = [
         sommet for sommet, data in graph.items()
         if data['numéro_ligne'] == ligne and data['si_terminus'] is True
     ]
   
     # Trouver le terminus le plus proche
     min_distance = float('inf')
     closest_terminus = None
   
     for terminus in terminus_candidates:
         # Calculer la distance depuis la station actuelle jusqu'au terminus
         distance_to_terminus = abs(distances[terminus] - distances[station_actuelle])
         if distance_to_terminus < min_distance:
             min_distance = distance_to_terminus
             closest_terminus = graph[terminus]['nom_sommet']
   
     return closest_terminus
def generate_instructions(graph: dict, chemin: list, distances: dict) -> list:
    """
    Génère les instructions pour le chemin donné.
    
    Parameters:
    - graph (dict): Dictionnaire représentant le graphe.
    - chemin (list): Liste des sommets du chemin calculé.
    - distances (dict): Dictionnaire des distances depuis le sommet de départ.
    
    Returns:
    - list: Instructions textuelles du chemin.
    """
    instructions = []
    current_line = None
    station_count = 0
    segment_time = 0
    first = True 
    nom = [graph[chemin[i]]['nom_sommet'] for i in range(len(chemin))]
    print("nom : ", nom)
    for i in range(len(chemin) - 1):
        current = chemin[i]
        next_station = chemin[i + 1]
        
        current_station = graph[current]
        next_station_data = graph[next_station]
        
        # Première station
        if i == 0:
            instructions.append(f"Vous êtes à {current_station['nom_sommet']}.")
            ref_station = current
        if current_station["nom_sommet"] == "Champs Élysées, Clémenceau": 
            print(i)
        # Changement de ligne ou continuation sur la même ligne
        if current_station['numéro_ligne'] != next_station_data['numéro_ligne']:
            # Si on change de ligne, on ajoute les informations du segment précédent
            if current_line is not None and first:
                #terminus_direction = find_correct_direction(graph, current, next_station, current_line, chemin[-1])
                terminus_direction = find_correct_direction2(graph, current, ref_station, current_line)
                instructions.append(
                    f"Prenez la ligne {current_line} direction {terminus_direction} pour {station_count} stations, "
                    f"durée estimée {segment_time // 60} minutes."
                )
                first = False
            elif current_line is not None :  
                terminus_direction = find_correct_direction2(graph, current, ref_station, current_line)
                # Ajouter l'instruction de changement de ligne
                instructions.append(
                    f"A {graph[ref_station]['nom_sommet']}, changez et prenez la ligne "
                    f"{current_line} direction {terminus_direction} pour {station_count} stations, durée estimée {segment_time // 60} minutes."
                )
            # Trouver la direction de la nouvelle ligne
            #terminus_direction = find_correct_direction(graph, next_station, chemin[-1], next_station_data['numéro_ligne'], chemin[-1])
            #terminus_direction = find_correct_direction2(graph, next_station, chemin[-1], next_station_data['numéro_ligne'])
            
            # Ajouter l'instruction de changement de ligne
            #instructions.append(
            #    f"A {current_station['nom_sommet']}, changez et prenez la ligne "
            #    f"{next_station_data['numéro_ligne']} direction {terminus_direction} pour {station_count} stations, durée estimée {segment_time // 60} minutes."
            #)
            
            # Réinitialiser les compteurs pour le nouveau segment de ligne
            ref_station = current
            current_line = next_station_data['numéro_ligne']
            station_count = 1
            segment_time = next(voisin['temps_en_secondes'] for voisin in current_station['voisins'] if voisin['num_sommet'] == next_station)
            print("segment_time : ", segment_time)
        else:
            # Si on reste sur la même ligne, on incrémente les compteurs
            if current_line is None:
                current_line = current_station['numéro_ligne']
            station_count += 1
            segment_time += next(voisin['temps_en_secondes'] for voisin in current_station['voisins'] if voisin['num_sommet'] == next_station)
    
    # Ajouter les informations pour le dernier segment de ligne
    terminus_direction = find_correct_direction(graph, chemin[-2], chemin[-1], current_line, chemin[-1])
    instructions.append(
        f"Prenez la ligne {current_line} direction {terminus_direction} pour {station_count} stations, durée estimée {segment_time // 60} minutes."
    )
    
    # Temps total pour le trajet complet
    instructions.append(f"Vous devriez arriver à {graph[chemin[-1]]['nom_sommet']} en environ {distances[chemin[-1]] // 60} minutes.")
    
    return instructions
def find_correct_direction(graph: dict, current_station: int, destination: int, ligne: int, terminus_target: int) -> str:
    """
    Trouve le terminus le plus proche sur une ligne donnée pour une station actuelle.
    
    Parameters:
    - graph (dict): Dictionnaire représentant le graphe.
    - current_station (int): Le sommet actuel.
    - destination (int): La station de destination.
    - ligne (int): Le numéro de la ligne.
    - terminus_target (int): Le terminus cible pour la direction.

    Returns:
    - str: Le nom du terminus le plus proche.
    """
    terminus_candidates = [
        sommet for sommet, data in graph.items()
        if data['numéro_ligne'] == ligne and data['si_terminus'] is True #and data['branchement'] == graph[destination]['branchement']
    ]
    if len (terminus_candidates) == 1:
        return graph[terminus_candidates[0]]['nom_sommet']

    min_distance_to_destination = float('inf')
    closest_terminus = None
    print("current_station : ", current_station)
    print("destination : ", destination)
    print("ligne : ", ligne)
    print("terminus_target : ", terminus_target)
    print('terminus_candidates : ', terminus_candidates)
    for terminus in terminus_candidates:
        print("terminus : ", terminus) 
        distance_from_current = count_stations_between(graph, current_station, terminus, ligne)
        print("distance_from_current : ", distance_from_current)
        distance_from_destination = count_stations_between(graph, destination, terminus, ligne)
        print("distance_from_destination : ", distance_from_destination)
        print("distance_from_destination < distance_from_current : ", distance_from_destination < distance_from_current)
        print("distance_from_destination < min_distance_to_destination : ", distance_from_destination < min_distance_to_destination)
        # Vérifier si le terminus est plus proche de la destination que de la station actuelle
        if distance_from_destination < distance_from_current and distance_from_destination < min_distance_to_destination:
            min_distance_to_destination = distance_from_destination
            print("min_distance_to_destination : ", min_distance_to_destination)
            closest_terminus = graph[terminus]['nom_sommet']
            print("closest_terminus : ", closest_terminus)
    
    return closest_terminus
